In [48]:
!pip install tensorflow-gpu==2.0.0-rc1 --quiet

     |████████████████████████████████| 380.5MB 39kB/s 
     |████████████████████████████████| 4.3MB 39.0MB/s 
     |████████████████████████████████| 51kB 6.1MB/s 
     |████████████████████████████████| 501kB 40.5MB/s 


In [46]:
%tensorflow_version 2.2

`%tensorflow_version` only switches the major version: 1.x or 2.x.
You set: `2.2`. This will be interpreted as: `2.x`.


TensorFlow is already loaded. Please restart the runtime to change versions.


## Keras and TensorFlow

In this assignment, we will learn about Keras and TensorFlow. We will create a neural network and measure the model's performance.

In [49]:
import numpy as np
import pandas as pd

In [50]:
cancer = pd.read_csv('https://tf-assets-prod.s3.amazonaws.com/tf-curric/data-science/breast_cancer.csv')

In [ ]:
cancer.head()

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,...,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,...,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,...,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,...,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,...,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


We'll start with some data cleaning. First remove the id column since we have no use for it. Then check for missing data and remove all rows containing missing data.

In [51]:
# Answer below:

cancer.drop("id", axis=1, inplace=True)


The diagnosis columnn is our target variable. How many possible values are in this variable?

In [52]:
# Answer below:

cancer.diagnosis.nunique()

2

Create one or multiple dummy variables for this column and store this encoded data in a numpy array called `target`

In [53]:
# Answer below:

target = pd.get_dummies(cancer.diagnosis,drop_first=False)
target.head(5)

,B,M
0,0,1
1,0,1
2,0,1
3,0,1
4,0,1


When looking at the columm names, you may have noticed that for each attribute we have a mean of the attribute, a standard error, and a worst value. Let's create a list of column names for only the means columns. Use this list to create a new dataframe containing only the means columns.

In [54]:
# Answer below:

cancer_mean = cancer.filter(like='_mean')
cancer_mean.head(5)

,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,fractal_dimension_mean
0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871
1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667
2,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999
3,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,0.09744
4,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,0.05883


Split the data into train and test samples. The test sample should contain 20% of the data.

In [67]:
# Answer below

from sklearn.model_selection import train_test_split
X = cancer_mean
y = target


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

Now we'll use the minmax scaler to ensure that all columns have values between zero and 1. Apply this transformations for all columns to be used in the prediction.

In [56]:
# Answer below:
from sklearn.preprocessing import MinMaxScaler

scale = MinMaxScaler()
X_train_scale = scale.fit_transform(X_train)
X_test_scale = scale.transform(X_test)


Now we can proceed with our model. In the cell below, import the `Sequential`, and `Dense` functions.

In [57]:
# Answer below:

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

create a model that takes the training data and predicts the target variable. The model will have 3 layers. The input layer should be of unit size 12 and the hidden layer will be of size 10. The output layer size will be determined by the size of the target variable. Use the sigmoid activation function for the last layer

In [58]:
# Answer below
model = Sequential()
model.add(Dense(12, input_shape=X_train.shape, activation='relu'))
model.add(Dense(10, activation='relu'))
model.add(Dense(1, activation='sigmoid'))


In [43]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_6 (Dense)              (None, 455, 12)           132       
_________________________________________________________________
dense_7 (Dense)              (None, 455, 10)           130       
_________________________________________________________________
dense_8 (Dense)              (None, 455, 1)            11        
Total params: 273
Trainable params: 273
Non-trainable params: 0
_________________________________________________________________


Compile and fit the model. Use the adam optimizer and the binary crossentropy loss function. Train the model for 200 epochs with a batch size of 80. Make sure touse the test data into the model as well.

In [62]:
import keras
from keras import layers

In [64]:
# Answer below:
model.compile(optimizer='Adam', loss='binary_crossentropy', 
              metrics=['accuracy'])


In [65]:
model.fit(X_train_scale, y_train, validation_data=(X_test_scale, y_test), batch_size=80, epochs=200, verbose=1)

Epoch 1/200


ValueError: ignored

Report the accuracy of the test sample.

In [ ]:
# Answer below:
score = model.evaluate(X_test_scale,y_test,verbose=0)
print('Test Score: ', score[0])
print('Test Accuracy: ', score[1])